In [1]:
import pandas as pd
import os
import numpy as np
from ast import literal_eval
import json
import pickle
import time

### Now we train (1st SVMs)

We use the for_train_test directory for this, so we create the datasets based on splits that were collected.

Note that the human datasets are not included in this repo and should be downloaded from the link to the original study. Preprocessing code can be found in the preprocessing notebook.

In [2]:
import pandas as pd
prompt_0 = pd.read_csv('gpt_datasets/for_train_test/prompt_h_0.csv').reset_index(drop=True)
prompt_gpt_0 = pd.read_csv('gpt_datasets/for_train_test/prompt_gpt_0.csv').reset_index(drop=True)

prompt_12 = pd.read_csv('gpt_datasets/for_train_test/prompt_h_12.csv').reset_index(drop=True)
taboo_12 = pd.read_csv('gpt_datasets/for_train_test/taboo_h_12.csv').reset_index(drop=True)
prompt_gpt_12 = pd.read_csv('gpt_datasets/for_train_test/prompt_gpt_12.csv').reset_index(drop=True)
taboo_gpt_12 = pd.read_csv('gpt_datasets/for_train_test/taboo_gpt_12.csv').reset_index(drop=True)
taboo_gpt_12_with_filt = pd.read_csv('gpt_datasets/for_train_test/taboo_gpt_12_filt.csv').reset_index(drop=True)

In [3]:
import re
import string
prompt_0['text']=prompt_0['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
prompt_gpt_0['text']=prompt_gpt_0['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

prompt_12['text']=prompt_12['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
taboo_12['text']=taboo_12['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
prompt_gpt_12['text']=prompt_gpt_12['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
taboo_gpt_12['text']=taboo_gpt_12['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
taboo_gpt_12_with_filt['text']=taboo_gpt_12_with_filt['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

In [4]:
import re
prompt_0['text']=prompt_0['text'].apply(lambda x: re.sub(' +',' ',x))
prompt_gpt_0['text']=prompt_gpt_0['text'].apply(lambda x: re.sub(' +',' ',x))

prompt_12['text']=prompt_12['text'].apply(lambda x: re.sub(' +',' ',x))
taboo_12['text']=taboo_12['text'].apply(lambda x: re.sub(' +',' ',x))
prompt_gpt_12['text']=prompt_gpt_12['text'].apply(lambda x: re.sub(' +',' ',x))
taboo_gpt_12['text']=taboo_gpt_12['text'].apply(lambda x: re.sub(' +',' ',x))
taboo_gpt_12_with_filt['text']=taboo_gpt_12_with_filt['text'].apply(lambda x: re.sub(' +',' ',x))

### below just change the train dataset and the test datasets and you are all good!

In [8]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from joblib import dump, load
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
import numpy as np
import itertools
from sklearn import svm
from sklearn.model_selection import train_test_split

same_res = []
same_f1 = []

same_gpt_res = []
same_gpt_f1 = []

taboo_res = []
taboo_f1 = []

taboo_gpt_res = []
taboo_gpt_f1 = []

for idx in range(0, 10):
    
    # first get test splits from 1 and 2; then combine the rest into train
    
    train_same, test_same = train_test_split(prompt_12, test_size=0.2)
    train_same = train_same.append(prompt_0)
    
    train_taboo, test_taboo = train_test_split(taboo_12, test_size=0.2)
    train_taboo = train_taboo.append(prompt_0)
    
    train_same_gpt, test_same_gpt = train_test_split(prompt_gpt_12, test_size=0.2)
    train_same_gpt = train_same_gpt.append(prompt_gpt_0)
    
    train_taboo_gpt, test_taboo_gpt = train_test_split(taboo_gpt_12, test_size=0.2) # can be exchanged for the split with tabooed samples
    train_taboo_gpt = train_taboo_gpt.append(prompt_gpt_0)
    
    pipeline = Pipeline([
            ('bow', CountVectorizer()),  
            ('tfidf', TfidfTransformer()),  
            ('c', svm.SVC(probability=True))
        ])
    fit = pipeline.fit(train_taboo['text'].str.lower(), train_taboo['label'])
    
    pred=pipeline.predict(test_same['text'].str.lower())
    same_res.append(accuracy_score(pred,test_same['label']))
    same_f1.append(f1_score(pred,test_same['label'],  average = 'weighted'))
    
    pred=pipeline.predict(test_same_gpt['text'].str.lower())
    same_gpt_res.append(accuracy_score(pred,test_same_gpt['label']))
    same_gpt_f1.append(f1_score(pred,test_same_gpt['label'],  average = 'weighted'))
        
    pred=pipeline.predict(test_taboo['text'].str.lower())
    taboo_res.append(accuracy_score(pred,test_taboo['label']))
    taboo_f1.append(f1_score(pred,test_taboo['label'],  average = 'weighted'))
        
    pred=pipeline.predict(test_taboo_gpt['text'].str.lower())
    taboo_gpt_res.append(accuracy_score(pred,test_taboo_gpt['label']))
    taboo_gpt_f1.append(f1_score(pred,test_taboo_gpt['label'],  average = 'weighted'))

In [9]:
arr = np.array(same_res)

print("ACC SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(same_f1)

print("F1 SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr))) 

arr = np.array(same_gpt_res)

print("ACC GPT SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(same_gpt_f1)

print("F1 GPT SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

ACC SAME MEAN: 0.9665364583333333 STD: 0.005647200115156623
F1 SAME MEAN: 0.9665461587404192 STD: 0.005619698126697387
ACC GPT SAME MEAN: 0.997872340425532 STD: 0.001822018805484418
F1 GPT SAME MEAN: 0.9978702053023847 STD: 0.0018244993888413507


In [10]:
arr = np.array(taboo_res)

print("ACC TABOO MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(taboo_f1)

print("F1 TABOO MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr))) 

arr = np.array(taboo_gpt_res)

print("ACC GPT TABOO MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(taboo_gpt_f1)

print("F1 GPT TABOO MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

ACC TABOO MEAN: 0.8397698209718669 STD: 0.012569103082045013
F1 TABOO MEAN: 0.8416977794559694 STD: 0.012121111024673689
ACC GPT TABOO MEAN: 0.9791831357048748 STD: 0.002751068908803825
F1 GPT TABOO MEAN: 0.9791853936117191 STD: 0.00274839346599952
